## Магические методы сравнений

eq - для равенства<br>
ne - для неравенства<br>
lt-  для оператора меньше<br>
le - для оператора меньше или равно<br>
gt - для оператора больше<br>
ge - для оператора больше или равно<br>

In [1]:
class Clock:
    __DAY = 24 * 60 * 60
    
    def __init__(self, seconds: int):
        if not isinstance(seconds, int):
            raise TypeError('Секунды должны быть целым числом')
        self.seconds = seconds % self.__DAY

Предположим, что существуют два экземпляра класса:

In [2]:
c1 = Clock(1000)
c2 = Clock(1000)

И мы хотим сравнить их:

In [3]:
c1 == c2

False

Оператор сработал, но сравнение объектов произошло по адресам объектов, на которые ссылаются переменные c1 и c2. Это стандартное поведение, но его можно переопределить, описав определенные магические методы в классе.

In [4]:
class Clock:
    __DAY = 24 * 60 * 60
    
    def __init__(self, seconds: int):
        if not isinstance(seconds, int):
            raise TypeError('Секунды должны быть целым числом')
        self.seconds = seconds % self.__DAY
        
    def __eq__(self, other):
        print('__eq__ bound method')
        if not isinstance(other, (int, Clock)):
            raise TypeError('Операнд справа может быть типа int или Clock')
        
        t = other if isinstance(other, int) else other.seconds
        
        return self.seconds == t

И теперь при сравнении двух объектов будет вызываться магический метод eq:

In [5]:
c1 = Clock(1000)
c2 = Clock(1000)

c1 == c2

__eq__ bound method


True

И также при сравнении с целым числом (все зависит от логики, прописанной в методе):

In [6]:
c1 == 1000

__eq__ bound method


True

Если прописан метод для равенства, но не прописан метод для неравенства, при попытке им воспользоваться:

In [7]:
c1 != 1200

__eq__ bound method


True

Будет вызван метод для равенства, так как неравенство - это отрицание равенства. Интерпретатор сам сделает отрицание, и все будет работать верно.

с1 != с2 - это все равно, что not (c1 == c2)

Метод lt (для оператора меньше) работает аналогичным образом:

In [8]:
class Clock:
    __DAY = 24 * 60 * 60
    
    def __init__(self, seconds: int):
        if not isinstance(seconds, int):
            raise TypeError('Секунды должны быть целым числом')
        self.seconds = seconds % self.__DAY
        
    def __eq__(self, other):
        print('__eq__ bound method')
        if not isinstance(other, (int, Clock)):
            raise TypeError('Операнд справа может быть типа int или Clock')
        
        t = other if isinstance(other, int) else other.seconds
        
        return self.seconds == t
    
    def __lt__(self, other):
        print('__lt__ bound method')
        if not isinstance(other, (int, Clock)):
            raise TypeError('Операнд справа может быть типа int или Clock')
        
        t = other if isinstance(other, int) else other.seconds
        
        return self.seconds < t

In [9]:
c1 = Clock(1000)
c2 = Clock(1200)

c1 < c2

__lt__ bound method


True

Также с помощью него будет работать оператор больше, если он не определен: интерпретатор будет менять операнды местами:

In [10]:
c1 > c2

__lt__ bound method


False

Код методов eq и lt во многом дублирован - поэтому его лучше перенести в отдельный метод:

In [11]:
class Clock:
    __DAY = 24 * 60 * 60
    
    def __init__(self, seconds: int):
        if not isinstance(seconds, int):
            raise TypeError('Секунды должны быть целым числом')
        self.seconds = seconds % self.__DAY
    
    @classmethod
    def __verify_data(cls, other):
        if not isinstance(other, (int, Clock)):
            raise TypeError('Операнд справа может быть типа int или Clock')
        
        return other if isinstance(other, int) else other.seconds
        
    def __eq__(self, other):
        print('__eq__ bound method')
        t = self.__verify_data(other)
        
        return self.seconds == t
    
    def __lt__(self, other):
        print('__lt__ bound method')
        t = self.__verify_data(other)
        
        return self.seconds < t

In [12]:
c1 = Clock(1000)
c2 = Clock(1200)

c1 < c2

__lt__ bound method


True

In [13]:
c1 > c2

__lt__ bound method


False

In [14]:
c1 == c2

__eq__ bound method


False

In [15]:
c1 != c2

__eq__ bound method


True

Методы le и ge работают по тому же принципу.

Для реализации операторов сравнения в классе достаточно определить по одному из пар методов, а другие сравнения будут работать через инверсию или замену операндов местами.